In [5]:
import unicodedata
import re
import json
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import acquire_dr
import prepare_dr

from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from time import strftime

In [10]:
df = pd.read_csv('raw_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       151 non-null    int64 
 1   repo             151 non-null    object
 2   language         143 non-null    object
 3   readme_contents  147 non-null    object
dtypes: int64(1), object(3)
memory usage: 4.8+ KB


In [18]:
df.language.value_counts()

Swift               65
Python              38
C++                 15
C                   11
JavaScript           2
Java                 2
Jupyter Notebook     2
HTML                 2
Dockerfile           1
Markdown             1
Shell                1
R                    1
Starlark             1
LLVM                 1
Name: language, dtype: int64

In [27]:
df[df.language.isnull()]

,Unnamed: 0,repo,language,readme_contents
0,0,apple/llvm-project,NaN,# Apple's fork of llvm-project\n\nThis is Appl...
13,13,apple/swift-docc-render-artifact,NaN,# Swift-DocC-Render-Artifact\n\nThis repositor...
14,14,apple/swift-lmdb,NaN,# CLMDB\n\nCLMDB is a SwiftPM package wrapper ...
83,83,apple/swift-community-hosted-continuous-integr...,NaN,# Swift Community-Hosted Continuous Integratio...
123,123,apple/ml-transcript-translation-consistency-ra...,NaN,# Human Ratings of Transcription/Translation C...
139,139,apple/llvm-monorepo-root,NaN,# Disclaimer\n\nThe [llvm-monorepo-root](https...
145,145,apple/swift-protobuf-plugin,NaN,"<img src=""https://swift.org/assets/images/swif..."
149,149,apple/swift-protobuf-test-conformance,NaN,"<img src=""https://swift.org/assets/images/swif..."


#### Observations:

- There are some repositories lacking a specified language, not sure how to proceed there but there's only 8 rows missing languages
    - an idea is to infer the language based on the repository title
- llvm project does not specify language
- swift docc render does not specify language but an idea would be to infer it's a majority swift repo?
- swift imdb is C, not sure why the acquire function didn't pick it up
- swift community hosted... does not specify language but an idea would be to infer it's a majority swift repo?
- ml transcript translation's repo looks like is just csv files
- llvm monorepo root is a wip repository for apple's open source Swift project.
- swift protobuf plugin was combined under the swift-protobuf repo, which is majority swift programming language
- swift protobuf test conformance was combined under the swift-protobuf repo, which is majority swift programming language

In [31]:
df[df.index == 83].repo

83    apple/swift-community-hosted-continuous-integr...
Name: repo, dtype: object

In [24]:
nan_languages = list(df[df.language.isnull()].index)

In [26]:
for i in nan_languages:
    display(df[df.index == i])

,Unnamed: 0,repo,language,readme_contents
0,0,apple/llvm-project,NaN,# Apple's fork of llvm-project\n\nThis is Appl...


,Unnamed: 0,repo,language,readme_contents
13,13,apple/swift-docc-render-artifact,NaN,# Swift-DocC-Render-Artifact\n\nThis repositor...


,Unnamed: 0,repo,language,readme_contents
14,14,apple/swift-lmdb,NaN,# CLMDB\n\nCLMDB is a SwiftPM package wrapper ...


,Unnamed: 0,repo,language,readme_contents
83,83,apple/swift-community-hosted-continuous-integr...,NaN,# Swift Community-Hosted Continuous Integratio...


,Unnamed: 0,repo,language,readme_contents
123,123,apple/ml-transcript-translation-consistency-ra...,NaN,# Human Ratings of Transcription/Translation C...


,Unnamed: 0,repo,language,readme_contents
139,139,apple/llvm-monorepo-root,NaN,# Disclaimer\n\nThe [llvm-monorepo-root](https...


,Unnamed: 0,repo,language,readme_contents
145,145,apple/swift-protobuf-plugin,NaN,"<img src=""https://swift.org/assets/images/swif..."


,Unnamed: 0,repo,language,readme_contents
149,149,apple/swift-protobuf-test-conformance,NaN,"<img src=""https://swift.org/assets/images/swif..."
